In [238]:
#Import Libraries


from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np





In [219]:
args={}
kwargs={}
args['batch_size']=1000
args['test_batch_size']=2
args['epochs']=10  #The number of Epochs is the number of times you go through the full dataset. 
args['lr']=0.01 #Learning rate is how fast it will decend. 
args['momentum']=0.5 #SGD momentum (default: 0.5) Momentum is a moving average of our gradients (helps to keep direction).

args['seed']=1 #random seed
args['log_interval']=10
args['cuda']=False


In [220]:
#load the data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['batch_size'], shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['test_batch_size'], shuffle=True, **kwargs)


In [139]:


class Net(nn.Module):
    #This defines the structure of the NN.
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()  #Dropout
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        #Convolutional Layer/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv1(x), 2)) 
        #Convolutional Layer/Dropout/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        #Fully Connected Layer/Activation
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        #Fully Connected Layer/Activation
        x = self.fc2(x)
        #Softmax gets probabilities. 
        return F.log_softmax(x, dim=1)


In [269]:

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        #Variables in Pytorch are differenciable. 
        data, target = Variable(data), Variable(target)
        #This will zero out the gradients for this batch. 
        optimizer.zero_grad()
        output = model(data)
        # Calculate the loss The negative log likelihood loss. It is useful to train a classification problem with C classes.
        loss = F.nll_loss(output, target)
        #dloss/dx for every Variable 
        loss.backward()
        #to do a one-step update on our parameter.
        optimizer.step()
        #Print out the loss periodically. 
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))

def test():
    with torch.no_grad():
      model.eval()
      test_loss = 0
      correct = 0
      for data, target in test_loader:
          if args['cuda']:
              data, target = data.cuda(), target.cuda()
          data, target = Variable(data), Variable(target)
          print(len(data), "Itmes")
          output = model(data)
          test_loss += F.nll_loss(output, target, size_average=False).data.item() # sum up batch loss
          pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
          correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

      test_loss /= len(test_loader.dataset)
      print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
          test_loss, correct, len(test_loader.dataset),
          100. * correct / len(test_loader.dataset)))



def testMultilabel():
    with torch.no_grad():
      model.eval()
      correct = 0
      newcorrect = 0

      for data, target in test_loader:
          oldtarget = target
          target = F.one_hot(target, num_classes=10)
          newdata = torch.zeros_like(data)[0:8]
          newtarget = torch.zeros_like(target, dtype=torch.float64)[0:8]
          
          for i in range(0, len(data), 2):
            x1 = data[i]
            x2 = data[i+1]
            y1 = target[i]
            y2 = target[i+1]
            x,y = concatImages(x1,x2,y1,y2)
            newdata[i//2] = x
            newtarget[i//2] = y



          # print("**********************INPUT: ")
          # print("Normal:")
          # printImageWithLabel(x1.numpy()[0], y1)
          # printImageWithLabel(x2.numpy()[0], y2)
          # print("MultiLabel: ")
          # printImageWithLabel(x.numpy()[0], y)
          output = model(data)
          newoutput = model(newdata)


          if args['cuda']:
            data, target = data.cuda(), target.cuda()
            data, target = Variable(data), Variable(target)
            newdata, newtarget = newdata.cuda(), newtarget.cuda()
            newdata, newtarget = Variable(newdata), Variable(newtarget)

          pred = output.data.max(1, keepdim=True)[1]
          correct += pred.eq(oldtarget.data.view_as(pred)).long().cpu().sum()

          
          answers = torch.argsort(newtarget, dim=1, descending=True).numpy()
          newpred = torch.argsort(newoutput, dim=1, descending=True).numpy()
          

          for i in range(0, len(data), 2):
            # print("preds     target       matches")
            # print(newpred[i//2][:2], "          ", answers[i//2][:2], "     ", compareArrays(answers[i//2][:2],newpred[i//2][:2]))
            newcorrect += compareArrays(answers[i//2][:2],newpred[i//2][:2])
          # correct += pred.eq(answers).long().cpu().sum()
          
          # for i in range(len(data)):
          #   print("Unipreds     Unitarget")
          #   print(pred[i].item(), "          ",oldtarget[i].item())
          # sleep(0)
          
          if correct % 1000 == 0 :
            print (correct)

          # correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()
      
      print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
          correct, len(test_loader.dataset),
          100. * correct / len(test_loader.dataset)))
      print('\nTest set: New Accuracy: {}/{} ({:.0f}%)\n'.format(
          newcorrect, len(test_loader.dataset),
          100. * newcorrect / len(test_loader.dataset)))






In [ ]:
model = Net()
if args['cuda']:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])

for epoch in range(1, args['epochs'] + 1):
    train(epoch)
    test()


In [271]:
testMultilabel()

tensor(2000)
tensor(3000)
tensor(4000)
tensor(6000)
tensor(7000)
tensor(9000)

Test set: Accuracy: 9299/10000 (93%)


Test set: New Accuracy: 2743/10000 (27%)



In [89]:
def concatImages(imageOne, imageTwo, labelOne, labelTwo, concatnitionAxis=2):
  x = torch.cat((imageOne, imageTwo), dim=concatnitionAxis)
  y = (labelOne+ labelTwo)/2

  temp = torch.from_numpy(cv2.resize(x.numpy()[0], (28, 28)) )
  x = torch.zeros_like(x1)
  x[0] = temp
  return x,y

In [6]:
def printImageWithLabel(image, label):
  fig, ax = plt.subplots()
  plt.imshow(image, cmap='gray')
  plt.show()
  print("Lable", label)

In [248]:
def compareArrays (a,b):
  matches = 0
  for i in range(len(a)):
    if a[i] in b :
      matches += 1
  return matches